### Purpose of the Code
This Python script processes low-quality images to highlight and extract bright (white-ish) areas, then saves the result.

#### Main Processing Steps
1. Load the Image
Reads an input image file (e.g., 1.png).

2. Noise Reduction & Grayscale Conversion
Applies Gaussian blur to reduce noise, then converts the image to grayscale.

3. Evaluate Noise Level
Calculates the standard deviation of the grayscale image as an indicator of how noisy the image is.

4. Adaptive Thresholding
Based on the noise level, the script adjusts thresholds (qerr1, qerr2) used to determine which pixels should be turned white.

5. Color Histogram Calculation
Computes histograms for red, green, and blue channels to find the most frequent color in each channel.

6. White Pixel Detection (Color Distance)
Compares each pixel’s color to the average color. If it’s close enough (within qerr1 or qerr2), the pixel is changed to white. This is done in two passes for stronger emphasis.

7. Binarization of Bright Areas
Uses thresholding to create a mask of bright regions, and forces those regions to white.

8. Save the Result
Saves the processed image in a folder called processed_images, with a filename like processed_1.png.

In [ ]:
import os
import cv2
import numpy as np
import math

def create_output_directory():
    output_dir = "processed_images"
    os.makedirs(output_dir, exist_ok=True)
    return output_dir

def load_image(filename):
    image = cv2.imread(filename)
    if image is None:
        raise FileNotFoundError(f"Error: Cannot open or read image file '{filename}'. Please check file path and integrity.")
    return image

def adaptive_threshold(image):
    """
    画質の悪い画像にも対応するため、適応的なしきい値を導入。
    - 画像のノイズが多い場合、しきい値を緩くする。
    - 明るい部分の判定を二値化で補助。
    
    To handle low-quality images, an adaptive thresholding approach is introduced.
    - When the image contains a lot of noise, the threshold is relaxed.
    - Bright areas are additionally identified using binarization.
    """
    height, width, _ = image.shape
    blurred = cv2.GaussianBlur(image, (5, 5), 0)  # Remove noise
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    
    # 画像全体の標準偏差を計算（ノイズレベルの指標）
    # Calculate the standard deviation of the entire image (as an indicator of noise level)
    std_dev = np.std(gray)
    
    # 標準偏差に応じてしきい値を調整
    # Adjust the threshold based on the standard deviation
    qerr1 = int(30 + (std_dev / 5))  # Increase the threshold as the standard deviation increases
    qerr2 = int(150 + (std_dev / 3))

    print(f"Adaptive Thresholds -> qerr1: {qerr1}, qerr2: {qerr2}, Std Dev: {std_dev:.2f}")

    # 明るい部分を二値化で取得
    # Extract bright areas using binarization
    _, bright_mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)

    # 色ヒストグラム計算
    # Calculate color histogram
    r_arr, g_arr, b_arr = np.zeros(256, dtype=int), np.zeros(256, dtype=int), np.zeros(256, dtype=int)
    
    for i in range(height):
        for j in range(width):
            b, g, r = image[i, j]
            r_arr[r] += 1
            g_arr[g] += 1
            b_arr[b] += 1
    
    r_mean = np.argmax(r_arr)
    g_mean = np.argmax(g_arr)
    b_mean = np.argmax(b_arr)

    print(f"Detected color mode -> R: {r_mean}, G: {g_mean}, B: {b_mean}")

    new_image = image.copy()

    # 明るさの距離が `qerr1` 以下なら白にする
    # Set to white if the brightness difference is less than or equal to `qerr1`
    for i in range(height):
        for j in range(width):
            b, g, r = new_image[i, j]
            err = math.sqrt((b - b_mean) ** 2 + (g - g_mean) ** 2 + (r - r_mean) ** 2)
            if err < qerr1:
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    # 明るさの距離が `qerr2` 以下なら白にする（強調）
    # Set to white if the brightness difference is less than or equal to `qerr2` (emphasized)
    for i in range(height):
        for j in range(width):
            b, g, r = new_image[i, j]
            err = math.sqrt((b - 255) ** 2 + (g - 255) ** 2 + (r - 255) ** 2)
            if err < qerr2:
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    # 二値化した明るい部分を適用
    # Apply the binarized bright areas
    for i in range(height):
        for j in range(width):
            if bright_mask[i, j] == 255:  # Force bright areas to be white
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    return new_image

def process_image(filename):
    try:
        output_dir = create_output_directory()
        image = load_image(filename)
        processed_image = adaptive_threshold(image)
        output_path = os.path.join(output_dir, "processed_" + os.path.basename(filename))
        
        if not cv2.imwrite(output_path, processed_image):
            raise IOError(f"Failed to save processed image to {output_path}")
        
        print(f"Processed image saved: {output_path}")
    
    except Exception as e:
        print(f"Error processing image '{filename}': {e}")

if __name__ == "__main__":
    filename = "1.png" 
    process_image(filename)

Adaptive Thresholds -> qerr1: 34, qerr2: 157, Std Dev: 23.85
Detected color mode -> R: 231, G: 216, B: 190
Processed image saved: processed_images/processed_1.png


In [1]:
import os
import cv2
import numpy as np
import math

def create_output_directory():
    output_dir = "processed_images"
    os.makedirs(output_dir, exist_ok=True)
    return output_dir

def load_image(filename):
    image = cv2.imread(filename)
    if image is None:
        raise FileNotFoundError(f"Error: Cannot open or read image file '{filename}'. Please check file path and integrity.")
    return image

def adaptive_threshold(image):
    height, width, _ = image.shape
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    
    std_dev = np.std(gray)
    qerr1 = int(30 + (std_dev / 5))
    qerr2 = int(150 + (std_dev / 3))

    print(f"Adaptive Thresholds -> qerr1: {qerr1}, qerr2: {qerr2}, Std Dev: {std_dev:.2f}")

    _, bright_mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)

    r_arr, g_arr, b_arr = np.zeros(256, dtype=int), np.zeros(256, dtype=int), np.zeros(256, dtype=int)
    for i in range(height):
        for j in range(width):
            b, g, r = image[i, j]
            r_arr[r] += 1
            g_arr[g] += 1
            b_arr[b] += 1

    r_mean = np.argmax(r_arr)
    g_mean = np.argmax(g_arr)
    b_mean = np.argmax(b_arr)

    print(f"Detected color mode -> R: {r_mean}, G: {g_mean}, B: {b_mean}")

    new_image = image.copy()

    for i in range(height):
        for j in range(width):
            b, g, r = new_image[i, j]
            err = math.sqrt((b - b_mean) ** 2 + (g - g_mean) ** 2 + (r - r_mean) ** 2)
            if err < qerr1:
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            b, g, r = new_image[i, j]
            err = math.sqrt((b - 255) ** 2 + (g - 255) ** 2 + (r - 255) ** 2)
            if err < qerr2:
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            if bright_mask[i, j] == 255:
                new_image[i, j] = np.array([255, 255, 255], dtype=np.uint8)

    return new_image

def process_all_images(input_dir="images", output_dir="processed_images"):
    os.makedirs(output_dir, exist_ok=True)
    supported_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(supported_extensions)]

    if not image_files:
        print(f"No image files found in '{input_dir}'")
        return

    for filename in image_files:
        try:
            filepath = os.path.join(input_dir, filename)
            image = load_image(filepath)
            processed_image = adaptive_threshold(image)
            output_path = os.path.join(output_dir, f"processed_{filename}")
            cv2.imwrite(output_path, processed_image)
            print(f"Saved: {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

if __name__ == "__main__":
    process_all_images()

Adaptive Thresholds -> qerr1: 37, qerr2: 162, Std Dev: 36.33
Detected color mode -> R: 224, G: 212, B: 183
Saved: processed_images/processed_J257..png
Adaptive Thresholds -> qerr1: 36, qerr2: 161, Std Dev: 34.80
Detected color mode -> R: 228, G: 215, B: 191
Saved: processed_images/processed_J241..png
Adaptive Thresholds -> qerr1: 37, qerr2: 162, Std Dev: 36.54
Detected color mode -> R: 244, G: 215, B: 187
Saved: processed_images/processed_J236..png
Adaptive Thresholds -> qerr1: 35, qerr2: 159, Std Dev: 28.22
Detected color mode -> R: 244, G: 211, B: 183
Saved: processed_images/processed_J237..png
Adaptive Thresholds -> qerr1: 37, qerr2: 161, Std Dev: 35.52
Detected color mode -> R: 219, G: 206, B: 175
Saved: processed_images/processed_J295..png
Adaptive Thresholds -> qerr1: 40, qerr2: 166, Std Dev: 50.14
Detected color mode -> R: 225, G: 213, B: 184
Saved: processed_images/processed_J240..png
Adaptive Thresholds -> qerr1: 38, qerr2: 164, Std Dev: 44.82
Detected color mode -> R: 218, G: